In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from scipy import stats, signal
import pandas as pd
import random

def set_seeds(seed=42):
    torch.manual_seed(seed)
    torch.mps.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seeds()

# Set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [120]:
class FeatureEnrichedDataset(Dataset):
    def __init__(self, X, y, session_ids, metadata, window_size=3):
        self.X = []
        self.y = []
        self.metadata = metadata
        self.session_ids = session_ids
        self.window_size = window_size

        # Group data by session
        session_groups = {}
        for idx, session_id in enumerate(session_ids):
            session_groups.setdefault(session_id, []).append((X[idx], y[idx]))

        # Create sliding windows within each session
        for session_id, session_data in session_groups.items():
            for i in range(len(session_data) - window_size + 1):
                steps = [data[0] for data in session_data[i:i+window_size]]
                window_X = np.vstack(steps)
                window_y = session_data[i+window_size-1][1]
                self.X.append(window_X)
                self.y.append(window_y)

        # Now extract features for all windows, then append duration
        self.X, self.y = self.extract_features()
    
    def extract_features(self):
        features = []
        labels = []
        for i in range(len(self.X)):
            X_win, y_val = self.X[i], self.y[i]
            sequence_features = []
            # Compute basic statistical features for each sensor
            for j in range(X_win.shape[1]):
                sensor_data = X_win[:, j]
                freqs, psd = signal.welch(sensor_data, fs=100)
                sequence_features.extend([
                    np.mean(sensor_data),
                    np.std(sensor_data),
                    np.min(sensor_data),
                    np.max(sensor_data),
                    np.median(sensor_data),
                    stats.skew(sensor_data),
                    stats.kurtosis(sensor_data),
                    np.percentile(sensor_data, 25),
                    np.percentile(sensor_data, 75),
                    np.ptp(sensor_data),
                    np.sum(psd),
                    np.mean(psd),
                    np.max(psd),
                    freqs[np.argmax(psd)]
                ])
            # Compute trend and dynamics features for each sensor
            for j in range(X_win.shape[1]):
                sensor_data = X_win[:, j]
                if len(sensor_data) > 5:
                    detrended = signal.detrend(sensor_data)
                    trend = sensor_data - detrended
                    sequence_features.append(np.mean(trend))
                    first_diff = np.diff(sensor_data)
                    sequence_features.extend([
                        np.mean(np.abs(first_diff)),
                        np.std(first_diff)
                    ])
                    if len(first_diff) > 1:
                        second_diff = np.diff(first_diff)
                        sequence_features.append(np.mean(np.abs(second_diff)))
            features.append(sequence_features)
            labels.append(y_val.item())
        
        # Append the session-level duration as a new feature for all samples
        features = self.add_mean_duration_feature(features)
        return np.array(features), np.array(labels)
    
    def add_mean_duration_feature(self, X_features):
        # Use the same number of session_ids as there are samples
        session_ids = self.session_ids[:len(X_features)]
        mean_durations = np.zeros(len(X_features))
        for session_id in np.unique(session_ids):
            session_mask = session_ids == session_id
            session_duration = self.metadata.loc[self.metadata['session_id'] == session_id, 'duration'].mean()
            mean_durations[session_mask] = session_duration
        mean_durations = mean_durations.reshape(-1, 1)
        return np.hstack((X_features, mean_durations))
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return (torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long))

In [121]:
class MLPModel(nn.Module):
    def __init__(self, input_shape, hidden_sizes=[256, 128, 64], dropout=0.4, num_classes=2):
        super(MLPModel, self).__init__()
        
        # Calculate flattened input size
        flattened_size = input_shape[0]
        
        # Build the hidden layers
        layers = []
        prev_size = flattened_size
        
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_size = hidden_size
        
        self.hidden_layers = nn.Sequential(*layers)
        self.output_layer = nn.Linear(hidden_sizes[-1], num_classes)
    
    def forward(self, x):
        # Flatten the input
        batch_size = x.size(0)
        x = x.view(batch_size, -1)
        
        # Pass through hidden layers
        x = self.hidden_layers(x)
        
        # Output layer
        x = self.output_layer(x)
        
        return x

In [122]:
# Define a feature-enriched LSTM model
class FeatureEnrichedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.3, num_classes=2):
        super(FeatureEnrichedLSTM, self).__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True, 
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )
        
        self.fc1 = nn.Linear(hidden_size*2, 64)
        self.dropout = nn.Dropout(dropout)
        self.bn = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, num_classes)
    
    def forward(self, x):
        print("x shape:", x.shape)
        
        # x shape: [batch_size, seq_len, input_size]
        
        # LSTM output: [batch_size, seq_len, hidden_size*2]
        lstm_out, _ = self.lstm(x)
        
        # Use the last time step output
        x = self.fc1(lstm_out[:, -1, :])
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [123]:
class CNN1D(nn.Module):
    def __init__(self, input_length, num_classes=2):
        super(CNN1D, self).__init__()
        # input_length is the length of the flattened feature vector from SensorDataset
        
        # Convolutional blocks
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.dropout1 = nn.Dropout(0.3)
        
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.dropout2 = nn.Dropout(0.3)
        
        self.skip_adjust = nn.Conv1d(64, 128, kernel_size=1)
        
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.bn3 = nn.BatchNorm1d(256)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.dropout3 = nn.Dropout(0.3)
        
        # Dynamically compute flattened size after convolutions using a dummy forward pass
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, input_length)  # shape: [batch, channels, length]
            dummy_out = self._forward_conv(dummy_input)
            self.flattened_size = dummy_out.view(1, -1).size(1)
        
        # Fully connected layers for classification
        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
    
    def _forward_conv(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x1 = self.pool1(x)
        x1 = self.dropout1(x1)
        
        skip = self.skip_adjust(x1)
        skip = F.max_pool1d(skip, kernel_size=2, stride=2, padding=1)
        
        x = self.conv2(x1)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        
        # Adjust skip connection if dimensions differ
        if skip.size(2) > x.size(2):
            x = x + skip[:, :, :x.size(2)]
        elif skip.size(2) < x.size(2):
            x = x[:, :, :skip.size(2)] + skip
        else:
            x = x + skip
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = self.dropout3(x)
        return x
    
    def forward(self, x):
        # x is of shape [batch_size, feature_length] from SensorDataset
        x = x.unsqueeze(1)  # now [batch_size, 1, feature_length]
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

In [124]:
# Training function
def train(model, train_loader, criterion, optimizer, device, epochs=50, scheduler=None):
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Print metrics for this epoch
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_train_loss:.4f}')
        
        # Update learning rate if scheduler is provided
        if scheduler is not None:
            scheduler.step(avg_train_loss)
    
    return model

In [125]:
# Evaluate the model
def evaluate(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            outputs = model(X_batch)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    
    print("\nModel Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds))

In [126]:
X_original = np.load('sensor_data.npy')  # Shape: (num_steps, time_steps, num_sensors)
metadata = pd.read_csv('combined_metadata.csv')
y = metadata['has_ms'].values

# Split by session for sequence integrity while stratifying
sessions = metadata['session_id'].values
unique_sessions = np.unique(sessions)

# Create a mapping of session_id to MS status
session_to_ms_status = {}
for session_id in unique_sessions:
    # Get all rows for this session
    session_mask = metadata['session_id'] == session_id
    # If any row has MS, the whole session is labeled as MS
    has_ms = any(metadata.loc[session_mask, 'has_ms'] == 1)
    session_to_ms_status[session_id] = 1 if has_ms else 0

# Create lists of session IDs by MS status
ms_sessions = [s for s, status in session_to_ms_status.items() if status == 1]
non_ms_sessions = [s for s, status in session_to_ms_status.items() if status == 0]

# Perform stratified split on MS and non-MS sessions separately
train_ms, temp_ms = train_test_split(ms_sessions, test_size=0.3, random_state=42, shuffle=True)
train_non_ms, temp_non_ms = train_test_split(non_ms_sessions, test_size=0.3, random_state=42, shuffle=True)

# Further split temp sets into validation and test
val_ms, test_ms = train_test_split(temp_ms, test_size=0.5, random_state=42, shuffle=True)
val_non_ms, test_non_ms = train_test_split(temp_non_ms, test_size=0.5, random_state=42, shuffle=True)

# Combine MS and non-MS sessions for each split
train_sessions = train_ms + train_non_ms
val_sessions = val_ms + val_non_ms
test_sessions = test_ms + test_non_ms

train_indices = metadata['session_id'].isin(train_sessions)
val_indices = metadata['session_id'].isin(val_sessions)
test_indices = metadata['session_id'].isin(test_sessions)

In [127]:
# Downsample X to have 10 timesteps instead of 100
X = X_original.reshape(X_original.shape[0], 25, -1, X_original.shape[2]).mean(axis=2)

X_train, X_val, X_test = X[train_indices], X[val_indices], X[test_indices]
y_train, y_val, y_test = y[train_indices], y[val_indices], y[test_indices]

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X.shape[2])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X.shape[2])).reshape(X_val.shape)

# Session IDs for reference
train_sessions_ids = sessions[train_indices]
val_sessions_ids = sessions[val_indices]

window_size = 4

# Code to set up and run the model
# Prepare feature-enriched datasets
train_dataset = FeatureEnrichedDataset(X_train, y_train, train_sessions_ids, metadata, window_size=window_size)
val_dataset = FeatureEnrichedDataset(X_val, y_val, val_sessions_ids, metadata, window_size=window_size)

# Prepare dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

/Users/talerez/Documents/AIPI_549/FinalProject/multiple-sclerosis-research/venv/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 100, using nperseg = 100
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


In [132]:
input_length = train_dataset[0][0].shape[0]
model = CNN1D(input_length=input_length, num_classes=2).to(device)

# Define loss function with class weighting
ms_weight = len(y_train) / (2 * np.sum(y_train == 1))
ms_weight = ms_weight * 0.9

weights = torch.tensor([1.0, ms_weight], dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss()

# Define optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)

# Train the model
print("Starting training...")
model = train(
    model, train_loader, 
    criterion, optimizer, device, 
    epochs=50, scheduler=scheduler
)

/Users/talerez/Documents/AIPI_549/FinalProject/multiple-sclerosis-research/venv/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting training...
Epoch 1/50, Loss: 0.4910
Epoch 2/50, Loss: 0.4255
Epoch 3/50, Loss: 0.4171
Epoch 4/50, Loss: 0.4102
Epoch 5/50, Loss: 0.4050
Epoch 6/50, Loss: 0.3971
Epoch 7/50, Loss: 0.3912
Epoch 8/50, Loss: 0.3890
Epoch 9/50, Loss: 0.3858
Epoch 10/50, Loss: 0.3803
Epoch 11/50, Loss: 0.3800
Epoch 12/50, Loss: 0.3750
Epoch 13/50, Loss: 0.3702
Epoch 14/50, Loss: 0.3634
Epoch 15/50, Loss: 0.3647
Epoch 16/50, Loss: 0.3552
Epoch 17/50, Loss: 0.3492
Epoch 18/50, Loss: 0.3487
Epoch 19/50, Loss: 0.3425
Epoch 20/50, Loss: 0.3430
Epoch 21/50, Loss: 0.3413
Epoch 22/50, Loss: 0.3395
Epoch 23/50, Loss: 0.3331
Epoch 24/50, Loss: 0.3278
Epoch 25/50, Loss: 0.3306
Epoch 26/50, Loss: 0.3311
Epoch 27/50, Loss: 0.3231
Epoch 28/50, Loss: 0.3265
Epoch 29/50, Loss: 0.3235
Epoch 30/50, Loss: 0.3244
Epoch 31/50, Loss: 0.3206
Epoch 32/50, Loss: 0.3184
Epoch 33/50, Loss: 0.3172
Epoch 34/50, Loss: 0.3230
Epoch 35/50, Loss: 0.3213
Epoch 36/50, Loss: 0.3138
Epoch 37/50, Loss: 0.3072
Epoch 38/50, Loss: 0.3173


In [133]:
# Evaluate the trained model
evaluate(model, val_loader, device)


Model Performance:
Accuracy: 0.8045
Precision: 0.7992
Recall: 0.6590
F1 Score: 0.7224

Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.90      0.85      1460
           1       0.80      0.66      0.72       918

    accuracy                           0.80      2378
   macro avg       0.80      0.78      0.79      2378
weighted avg       0.80      0.80      0.80      2378



Model Performance:
Accuracy: 0.8389
Precision: 0.8078
Recall: 0.7647
F1 Score: 0.7857

In [ ]:
input_size = train_dataset[0][0].shape[0]
model = MLPModel(input_shape=(input_size,)).to(device)

# Set up optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()#weight=weights)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# Train with your existing function
model = train(model, train_loader, criterion, optimizer, device, epochs=75, scheduler=scheduler)

/Users/talerez/Documents/AIPI_549/FinalProject/multiple-sclerosis-research/venv/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/75, Loss: 0.5014
Epoch 2/75, Loss: 0.4158
Epoch 3/75, Loss: 0.3861
Epoch 4/75, Loss: 0.3623
Epoch 5/75, Loss: 0.3484
Epoch 6/75, Loss: 0.3361
Epoch 7/75, Loss: 0.3266
Epoch 8/75, Loss: 0.3206
Epoch 9/75, Loss: 0.3102
Epoch 10/75, Loss: 0.3059
Epoch 11/75, Loss: 0.3036
Epoch 12/75, Loss: 0.3072
Epoch 13/75, Loss: 0.3027
Epoch 14/75, Loss: 0.2983
Epoch 15/75, Loss: 0.3026
Epoch 16/75, Loss: 0.3021
Epoch 17/75, Loss: 0.2935
Epoch 18/75, Loss: 0.2999
Epoch 19/75, Loss: 0.2966
Epoch 20/75, Loss: 0.2966
Epoch 21/75, Loss: 0.2838
Epoch 22/75, Loss: 0.2885
Epoch 23/75, Loss: 0.2923
Epoch 24/75, Loss: 0.2945
Epoch 25/75, Loss: 0.2892
Epoch 26/75, Loss: 0.2885
Epoch 27/75, Loss: 0.2851
Epoch 28/75, Loss: 0.2774
Epoch 29/75, Loss: 0.2798
Epoch 30/75, Loss: 0.2684
Epoch 31/75, Loss: 0.2767
Epoch 32/75, Loss: 0.2686
Epoch 33/75, Loss: 0.2673
Epoch 34/75, Loss: 0.2680
Epoch 35/75, Loss: 0.2751
Epoch 36/75, Loss: 0.2701
Epoch 37/75, Loss: 0.2677
Epoch 38/75, Loss: 0.2684
Epoch 39/75, Loss: 0.

In [107]:
# Evaluate the trained model
evaluate(model, val_loader, device)


Model Performance:
Accuracy: 0.7843
Precision: 0.9571
Recall: 0.4619
F1 Score: 0.6231

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.99      0.85      1460
           1       0.96      0.46      0.62       918

    accuracy                           0.78      2378
   macro avg       0.85      0.72      0.74      2378
weighted avg       0.83      0.78      0.76      2378



Model Performance:
Accuracy: 0.8192
Precision: 0.8058
Recall: 0.7004
F1 Score: 0.7494

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1460
           1       0.81      0.70      0.75       918

    accuracy                           0.82      2378
   macro avg       0.82      0.80      0.80      2378
weighted avg       0.82      0.82      0.82      2378